In [ ]:
import music
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from reformattedDecisionTree import DecisionTreeClassifier
from reformattedDecisionTree import Node
import RandomForest


In [ ]:
#import GPUtil
#GPUtil.getAvailable()

In [ ]:

data= music.get_music()
art_key=list(data[0]['artist'].keys())
art_key[1]='artist_hotttnesss'   #rename some of the header which has same name
art_key[2]='artist_id'
art_key[6]='artist_name'
song_key=list(data[0]['song'].keys())
song_key[8]='song_hotttnesss'
song_key[9]='song_id'
rel_key=list(data[0]['release'].keys())
rel_key[0]='release_id'
rel_key[1]='release_name'
headers=art_key+rel_key+song_key
all_row=[]
for i in range(len(data)):
    art_val=list(data[i]['artist'].values())
    rel_val=list(data[i]['release'].values())
    song_val=list(data[i]['song'].values())
    each_row=art_val+rel_val+song_val
    all_row.append(each_row)
    
filename = 'song.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file) 
    csvwriter.writerow(headers)
    csvwriter.writerows(all_row)
    
file_path = 'song.csv'
df = pd.read_csv(file_path)
print(df)

In [ ]:
 #remove zero-number column

columns_to_remove = ['location', 'similar', 'mode','title','latitude','longitude','artist_id','release_id','release_name','song_id','year']

    # Removing multiple columns
df = df.drop(columns=columns_to_remove, axis=1)

    # Display the DataFrame after removing columns
print(df)

text_columns = []  # List to store columns with text data

    # Iterate through columns and check data types
for column in df.columns:
        if df[column].dtype == object:  # Check if the data type is 'object' (usually represents text)
            text_columns.append(column)  # Add the column name to the list
            #print(column)

In [ ]:
  #mapping terms and artist_name to a unique machine readable value and throw the old string column
label_encoder = LabelEncoder()
df['terms_encode'] = label_encoder.fit_transform(df['terms'])
df['artist_name_encode'] = label_encoder.fit_transform(df['artist_name'])
columns_to_remove=['terms','artist_name']
df = df.drop(columns=columns_to_remove, axis=1)
print(df)



In [ ]:
augmented_df = pd.concat([df] * 10, ignore_index=True)  # Duplicate DataFrame ten times
noise = np.random.normal(0.05, 0.02, size=(len(augmented_df), len(augmented_df.columns)))  # Generate Gaussian noise 
augmented_df = augmented_df + noise  # Add noise to the DataFrame

augmented_df['familiarity_label'] = augmented_df['familiarity'].apply(lambda x: 1 if x > 0.6 else 0) # add labeled column and name it as familarity_label
columns_to_remove=['familiarity']
augmented_df = augmented_df.drop(columns=columns_to_remove, axis=1)
print(augmented_df)

X = augmented_df.drop(['familiarity_label'], axis=1)
Y = augmented_df['familiarity_label']



## Random-forest

In [16]:
!brew update
!arch -arm64 brew install  libomp

Already up-to-date.
libomp  is already installed but outdated (so it will be upgraded).
==> Downloading https://ghcr.io/v2/homebrew/core/libomp/manifests/17.0.6
######################################################################### 100.0%
==> Fetching libomp
==> Downloading https://ghcr.io/v2/homebrew/core/libomp/blobs/sha256:1a771c79450
######################################################################### 100.0%
==> Pouring libomp--17.0.6.arm64_monterey.bottle.tar.gz
==> Caveats
libomp is keg-only, which means it was not symlinked into /opt/homebrew,
because it can override GCC headers and result in broken builds.

For compilers to find libomp you may need to set:
  export LDFLAGS="-L/opt/homebrew/opt/libomp/lib"
  export CPPFLAGS="-I/opt/homebrew/opt/libomp/include"
==> Summary
🍺  /opt/homebrew/Cellar/libomp/17.0.6: 7 files, 1.7MB
==> Running `brew cleanup libomp`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see

In [ ]:
#parameter
features= augmented_df.iloc[:500, :-1]
target= augmented_df.iloc[:500, -1]
X_train, x_test, Y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
print(Y_train.shape)
model = RandomForest.random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
preds = predict_rf(model, X_test)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, pre)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='coolwarm')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [12]:
# y_pred=classifier.predict(x_test)

In [18]:
#parameter
features= augmented_df.iloc[:100, :-1]
target= augmented_df.iloc[:100, -1]
X_train, x_test, Y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
print(Y_train.shape)
model = RandomForest.random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
preds = predict_rf(model, X_test)

(80,)


KeyboardInterrupt: 

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load sample data (replace this with your own data)
data = load_iris()
X = augmented_df.iloc[:500, :-1]
y = augmented_df.iloc[:500, -1]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using LazyClassifier with Random Forest
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Print the models and their performance
print(models)

# Example of getting the confusion matrix for Random Forest
rf_predictions = predictions['RandomForestClassifier']
conf_matrix = confusion_matrix(y_test, rf_predictions)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Random Forest')
plt.show()